In [33]:
# basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# model
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score, recall_score, precision_score, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import shap

import model_pipeline

In [34]:
ohe_mode_allcat  = pd.read_csv("data/ohe_mode_allcat.csv")

In [35]:
ohe_mode_allcat.drop(columns=['Unnamed: 0'], inplace=True)

In [38]:
for i in ohe_mode_allcat.columns:
    print(i)

destination_Home
destination_No Urgent Place
destination_Work
passanger_Alone
passanger_Friend(s)
passanger_Kid(s)
passanger_Partner
weather_Rainy
weather_Snowy
weather_Sunny
temperature_30
temperature_55
temperature_80
time_10AM
time_10PM
time_2PM
time_6PM
time_7AM
coupon_Bar
coupon_Carry out & Take away
coupon_Coffee House
coupon_Restaurant(20-50)
coupon_Restaurant(<20)
expiration_1d
expiration_2h
gender_Female
gender_Male
age_21
age_26
age_31
age_36
age_41
age_46
age_50plus
age_below21
maritalStatus_Divorced
maritalStatus_Married partner
maritalStatus_Single
maritalStatus_Unmarried partner
maritalStatus_Widowed
has_children_0
has_children_1
education_Associates degree
education_Bachelors degree
education_Graduate degree (Masters or Doctorate)
education_High School Graduate
education_Some High School
education_Some college - no degree
occupation_Architecture & Engineering
occupation_Arts Design Entertainment Sports & Media
occupation_Building & Grounds Cleaning & Maintenance
occupati

In [37]:
ohe_mode_allcat.head()

,destination_Home,destination_No Urgent Place,destination_Work,passanger_Alone,passanger_Friend(s),passanger_Kid(s),passanger_Partner,weather_Rainy,weather_Snowy,weather_Sunny,...,toCoupon_GEQ5min_1,toCoupon_GEQ15min_0,toCoupon_GEQ15min_1,toCoupon_GEQ25min_0,toCoupon_GEQ25min_1,direction_same_0,direction_same_1,direction_opp_0,direction_opp_1,Y
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0


In [29]:
mode_allcat = pd.read_csv("data/mode_allcat.csv")
mode_ordinal = pd.read_csv("data/mode_ordinal.csv")
nan_allcat = pd.read_csv("data/nan_allcat.csv")

In [39]:
# change column name for xgboost
mode_allcat = mode_allcat.rename(columns={'coupon_Restaurant(<20)':'coupon_Restaurant(less_than_20)'})
mode_ordinal = mode_ordinal.rename(columns={'coupon_Restaurant(<20)':'coupon_Restaurant(less_than_20)'})
nan_allcat = nan_allcat.rename(columns={'coupon_Restaurant(<20)':'coupon_Restaurant(less_than_20)'})
ohe_mode_allcat  = ohe_mode_allcat.rename(columns={'coupon_Restaurant(<20)':'coupon_Restaurant(less_than_20)'})

In [21]:
mode_ordinal.shape

(12684, 89)

In [22]:
datasets = [("mode_allcat",mode_allcat),("mode_ordinal",mode_ordinal),('nan_allcat',nan_allcat)]

In [40]:
dt = DecisionTreeClassifier()
dt_params = {"max_depth":[10], "min_samples_leaf":[12], "min_samples_split":[80], "random_state":[7], "splitter":['random']}

rf = RandomForestClassifier()
rf_params= {"max_depth":[8], "min_samples_leaf":[7], "min_samples_split":[8], "n_estimators":[500], "random_state":[7]}

xgb = XGBClassifier()
xgb_params = {"eta":[.3],"gamma":[1],"max_depth":[7],"lambda":[0], "alpha":[1]}

nn = MLPClassifier()
nn_params = {"hidden_layer_sizes":[(400,)], "learning_rate_init":[0.01], "max_iter":[500],
              "random_state":[7], "shuffle":[False]}

In [41]:
res = []
key_modified_params = {f"model__{key}": val for key, val in xgb_params.items()}
output = model_pipeline.imb_pipe_fit(xgb, key_modified_params, 'ohe_mode_allcat', ohe_mode_allcat.drop(columns='Y'), ohe_mode_allcat["Y"], score='roc_auc', scaler=True)
res.append(output)

fitting model...


In [21]:
# refit model for decision tree
res = []
for data in datasets:
    key_modified_params = {f"model__{key}": val for key, val in dt_params.items()}
    output = model_pipeline.imb_pipe_fit(dt, key_modified_params, data[0], data[1].drop(columns='Y'), data[1]["Y"], score='roc_auc', scaler=True)
    res.append(output)

fitting model...
fitting model...
fitting model...


In [14]:
res[0]['test_score']

0.8415518780992912

In [24]:
res[0]['test_score'],res[1]['test_score'],res[2]['test_score']

(0.7307617527660086, 0.7372665438033172, 0.7382605336322586)

In [26]:
# refit model for random forest
res = []
for data in datasets:
    key_modified_params = {f"model__{key}": val for key, val in rf_params.items()}
    output = model_pipeline.imb_pipe_fit(rf, key_modified_params, data[0], data[1].drop(columns='Y'), data[1]["Y"], score='roc_auc', scaler=True)
    res.append(output)

fitting model...
fitting model...
fitting model...


In [27]:
res[0]['test_score'],res[1]['test_score'],res[2]['test_score']

(0.7645342909074788, 0.7617755653930678, 0.7625203452840108)

In [29]:
# refit model for xgboost
res = []
for data in datasets:
    key_modified_params = {f"model__{key}": val for key, val in xgb_params.items()}
    output = model_pipeline.imb_pipe_fit(xgb, key_modified_params, data[0], data[1].drop(columns='Y'), data[1]["Y"], score='roc_auc', scaler=True)
    res.append(output)

fitting model...
fitting model...
fitting model...


In [30]:
res[0]['test_score'],res[1]['test_score'],res[2]['test_score']

(0.8337506887314042, 0.8320825337715881, 0.8334371971956758)

In [36]:
# refit model for neural network
res = []
for data in datasets:
    key_modified_params = {f"model__{key}": val for key, val in nn_params.items()}
    output = model_pipeline.imb_pipe_fit(nn, key_modified_params, data[0], data[1].drop(columns='Y'), data[1]["Y"], score='roc_auc', scaler=True)
    res.append(output)

fitting model...
fitting model...
fitting model...


In [37]:
res[0]['test_score'],res[1]['test_score'],res[2]['test_score']

(0.799376183509712, 0.7732740549338503, 0.7765558996573759)

In [39]:
d = {'dataset':["mode_allcat","mode_ordinal","nan_allcat"],
     'Decision_tree':[0.7307617527660086, 0.7372665438033172, 0.7382605336322586],
     'Random_Forest':[0.7645342909074788, 0.7617755653930678, 0.7625203452840108],
     "Xgboost":[0.8337506887314042, 0.8320825337715881, 0.8334371971956758],
     "MLP_classifier":[0.799376183509712, 0.7732740549338503, 0.7765558996573759],
     }
results = pd.DataFrame(d)


In [40]:
results

,dataset,Decision_tree,Random_Forest,Xgboost,MLP_classifier
0,mode_allcat,0.730762,0.764534,0.833751,0.799376
1,mode_ordinal,0.737267,0.761776,0.832083,0.773274
2,nan_allcat,0.738261,0.762520,0.833437,0.776556
